##Zimnat Insurance Recommendation Challenge
Can you predict which insurance products existing clients will want next?
###Baseline with lgbm

### Get data

In [247]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import datetime
import matplotlib.pyplot as plt
import seaborn as sns


In [248]:
pd.set_option('display.max_columns', 30000)
pd.set_option('display.max_row', 30000)

In [249]:
train = pd.read_csv('Train.csv',  
                  # parse_dates=['join_date'],
                # dayfirst=True
                   )
test = pd.read_csv('Test.csv',  
                # parse_dates=['join_date'],
                #  dayfirst=True
                  )
submission = pd.read_csv('SampleSubmission.csv')

In [250]:
train.marital_status.replace('f', "F", inplace=True) 

In [251]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 29 columns):
ID                          29132 non-null object
join_date                   29130 non-null object
sex                         29132 non-null object
marital_status              29132 non-null object
birth_year                  29132 non-null int64
branch_code                 29132 non-null object
occupation_code             29132 non-null object
occupation_category_code    29132 non-null object
P5DA                        29132 non-null int64
RIBP                        29132 non-null int64
8NN1                        29132 non-null int64
7POT                        29132 non-null int64
66FJ                        29132 non-null int64
GYSR                        29132 non-null int64
SOP4                        29132 non-null int64
RVSZ                        29132 non-null int64
PYUQ                        29132 non-null int64
LJR9                        29132 non-null int64
N2

# Some new features

In [252]:
#Master
means = train.groupby('branch_code')['P5DA'].mean()
train['branch_code_P5DA'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['RIBP'].mean()
train['branch_code_RIBP'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['8NN1'].mean()
train['branch_code_8NN1'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['7POT'].mean()
train['branch_code_7POT'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['66FJ'].mean()
train['branch_code_66FJ'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['GYSR'].mean()
train['branch_code_GYSR'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['SOP4'].mean()
train['branch_code_SOP4'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['RVSZ'].mean()
train['branch_code_RVSZ'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['PYUQ'].mean()
train['branch_code_PYUQ'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['LJR9'].mean()
train['branch_code_LJR9'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['N2MW'].mean()
train['branch_code_N2MW'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['AHXO'].mean()
train['branch_code_AHXO'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['BSTQ'].mean()
train['branch_code_BSTQ'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['FM3X'].mean()
train['branch_code_FM3X'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['K6QO'].mean()
train['branch_code_K6QO'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['QBOL'].mean()
train['branch_code_QBOL'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['JWFN'].mean()
train['branch_code_JWFN'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['JZ9D'].mean()
train['branch_code_JZ9D'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['J9JW'].mean()
train['branch_code_J9JW'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['GHYX'].mean()
train['branch_code_GHYX'] = train['branch_code'].map(means)

means = train.groupby('branch_code')['ECY3'].mean()
train['branch_code_ECY3'] = train['branch_code'].map(means)

#test

means = train.groupby('branch_code')['P5DA'].mean()
test['branch_code_P5DA'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['RIBP'].mean()
test['branch_code_RIBP'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['8NN1'].mean()
test['branch_code_8NN1'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['7POT'].mean()
test['branch_code_7POT'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['66FJ'].mean()
test['branch_code_66FJ'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['GYSR'].mean()
test['branch_code_GYSR'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['SOP4'].mean()
test['branch_code_SOP4'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['RVSZ'].mean()
test['branch_code_RVSZ'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['PYUQ'].mean()
test['branch_code_PYUQ'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['LJR9'].mean()
test['branch_code_LJR9'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['N2MW'].mean()
test['branch_code_N2MW'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['AHXO'].mean()
test['branch_code_AHXO'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['BSTQ'].mean()
test['branch_code_BSTQ'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['FM3X'].mean()
test['branch_code_FM3X'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['K6QO'].mean()
test['branch_code_K6QO'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['QBOL'].mean()
test['branch_code_QBOL'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['JWFN'].mean()
test['branch_code_JWFN'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['JZ9D'].mean()
test['branch_code_JZ9D'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['J9JW'].mean()
test['branch_code_J9JW'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['GHYX'].mean()
test['branch_code_GHYX'] = test['branch_code'].map(means)

means = train.groupby('branch_code')['ECY3'].mean()
test['branch_code_ECY3'] = test['branch_code'].map(means)


In [253]:
means = train.groupby('sex')['P5DA'].mean()
train['sex_P5DA'] = train['sex'].map(means)

means = train.groupby('sex')['RIBP'].mean()
train['sex_RIBP'] = train['sex'].map(means)

means = train.groupby('sex')['8NN1'].mean()
train['sex_8NN1'] = train['sex'].map(means)

means = train.groupby('sex')['7POT'].mean()
train['sex_7POT'] = train['sex'].map(means)

means = train.groupby('sex')['66FJ'].mean()
train['sex_66FJ'] = train['sex'].map(means)

means = train.groupby('sex')['GYSR'].mean()
train['sex_GYSR'] = train['sex'].map(means)

means = train.groupby('sex')['SOP4'].mean()
train['sex_SOP4'] = train['sex'].map(means)

means = train.groupby('sex')['RVSZ'].mean()
train['sex_RVSZ'] = train['sex'].map(means)

means = train.groupby('sex')['PYUQ'].mean()
train['sex_PYUQ'] = train['sex'].map(means)

means = train.groupby('sex')['LJR9'].mean()
train['sex_LJR9'] = train['sex'].map(means)

means = train.groupby('sex')['N2MW'].mean()
train['sex_N2MW'] = train['sex'].map(means)

means = train.groupby('sex')['AHXO'].mean()
train['sex_AHXO'] = train['sex'].map(means)

means = train.groupby('sex')['BSTQ'].mean()
train['sex_BSTQ'] = train['sex'].map(means)

means = train.groupby('sex')['FM3X'].mean()
train['sex_FM3X'] = train['sex'].map(means)

means = train.groupby('sex')['K6QO'].mean()
train['sex_K6QO'] = train['sex'].map(means)

means = train.groupby('sex')['QBOL'].mean()
train['sex_QBOL'] = train['sex'].map(means)

means = train.groupby('sex')['JWFN'].mean()
train['sex_JWFN'] = train['sex'].map(means)

means = train.groupby('sex')['JZ9D'].mean()
train['sex_JZ9D'] = train['sex'].map(means)

means = train.groupby('sex')['J9JW'].mean()
train['sex_J9JW'] = train['sex'].map(means)

means = train.groupby('sex')['GHYX'].mean()
train['sex_GHYX'] = train['sex'].map(means)

means = train.groupby('sex')['ECY3'].mean()
train['sex_ECY3'] = train['sex'].map(means)

#test

means = train.groupby('sex')['P5DA'].mean()
test['sex_P5DA'] = test['sex'].map(means)

means = train.groupby('sex')['RIBP'].mean()
test['sex_RIBP'] = test['sex'].map(means)

means = train.groupby('sex')['8NN1'].mean()
test['sex_8NN1'] = test['sex'].map(means)

means = train.groupby('sex')['7POT'].mean()
test['sex_7POT'] = test['sex'].map(means)

means = train.groupby('sex')['66FJ'].mean()
test['sex_66FJ'] = test['sex'].map(means)

means = train.groupby('sex')['GYSR'].mean()
test['sex_GYSR'] = test['sex'].map(means)

means = train.groupby('sex')['SOP4'].mean()
test['sex_SOP4'] = test['sex'].map(means)

means = train.groupby('sex')['RVSZ'].mean()
test['sex_RVSZ'] = test['sex'].map(means)

means = train.groupby('sex')['PYUQ'].mean()
test['sex_PYUQ'] = test['sex'].map(means)

means = train.groupby('sex')['LJR9'].mean()
test['sex_LJR9'] = test['sex'].map(means)

means = train.groupby('sex')['N2MW'].mean()
test['sex_N2MW'] = test['sex'].map(means)

means = train.groupby('sex')['AHXO'].mean()
test['sex_AHXO'] = test['sex'].map(means)

means = train.groupby('sex')['BSTQ'].mean()
test['sex_BSTQ'] = test['sex'].map(means)

means = train.groupby('sex')['FM3X'].mean()
test['sex_FM3X'] = test['sex'].map(means)

means = train.groupby('sex')['K6QO'].mean()
test['sex_K6QO'] = test['sex'].map(means)

means = train.groupby('sex')['QBOL'].mean()
test['sex_QBOL'] = test['sex'].map(means)

means = train.groupby('sex')['JWFN'].mean()
test['sex_JWFN'] = test['sex'].map(means)

means = train.groupby('sex')['JZ9D'].mean()
test['sex_JZ9D'] = test['sex'].map(means)

means = train.groupby('sex')['J9JW'].mean()
test['sex_J9JW'] = test['sex'].map(means)

means = train.groupby('sex')['GHYX'].mean()
test['sex_GHYX'] = test['sex'].map(means)

means = train.groupby('sex')['ECY3'].mean()
test['sex_ECY3'] = test['sex'].map(means)


In [254]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,branch_code_P5DA,branch_code_RIBP,branch_code_8NN1,branch_code_7POT,branch_code_66FJ,branch_code_GYSR,branch_code_SOP4,branch_code_RVSZ,branch_code_PYUQ,branch_code_LJR9,branch_code_N2MW,branch_code_AHXO,branch_code_BSTQ,branch_code_FM3X,branch_code_K6QO,branch_code_QBOL,branch_code_JWFN,branch_code_JZ9D,branch_code_J9JW,branch_code_GHYX,branch_code_ECY3,sex_P5DA,sex_RIBP,sex_8NN1,sex_7POT,sex_66FJ,sex_GYSR,sex_SOP4,sex_RVSZ,sex_PYUQ,sex_LJR9,sex_N2MW,sex_AHXO,sex_BSTQ,sex_FM3X,sex_K6QO,sex_QBOL,sex_JWFN,sex_JZ9D,sex_J9JW,sex_GHYX,sex_ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.006907,0.000000,0.001535,0.000767,0.000000,0.000000,0.990790,0.002302,0.000000,0.002302,0.000000,0.001535,0.000000,0.966999,0.052955,0.002302,0.003070,0.003070,0.023024,0.010744,0.001931,0.039873,0.005453,0.010905,0.008065,0.000000,0.008747,0.847779,0.078269,0.012950,0.027036,0.016585,0.009997,0.003522,0.742929,0.239237,0.011587,0.066455,0.065887,0.046121,0.029422
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.009125,0.000855,0.002566,0.000855,0.000000,0.003137,0.858283,0.059025,0.006843,0.004562,0.001141,0.003137,0.000000,0.777873,0.218420,0.002281,0.112917,0.112632,0.019390,0.021101,0.001931,0.039873,0.005453,0.010905,0.008065,0.000000,0.008747,0.847779,0.078269,0.012950,0.027036,0.016585,0.009997,0.003522,0.742929,0.239237,0.011587,0.066455,0.065887,0.046121,0.029422
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000369,0.143314,0.009710,0.017822,0.026794,0.000123,0.030605,0.887045,0.066740,0.017085,0.010447,0.001967,0.014503,0.005162,0.716445,0.238569,0.003073,0.024705,0.024705,0.031957,0.027040,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.006907,0.000000,0.001535,0.000767,0.000000,0.000000,0.990790,0.002302,0.000000,0.002302,0.000000,0.001535,0.000000,0.966999,0.052955,0.002302,0.003070,0.003070,0.023024,0.010744,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0.000369,0.143314,0.009710,0.017822,0.026794,0.000123,0.030605,0.887045,0.066740,0.017085,0.010447,0.001967,0.014503,0.005162,0.716445,0.238569,0.003073,0.024705,0.024705,0.031957,0.027040,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468


In [255]:
train.groupby('sex').sum()

,birth_year,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,branch_code_P5DA,branch_code_RIBP,branch_code_8NN1,branch_code_7POT,branch_code_66FJ,branch_code_GYSR,branch_code_SOP4,branch_code_RVSZ,branch_code_PYUQ,branch_code_LJR9,branch_code_N2MW,branch_code_AHXO,branch_code_BSTQ,branch_code_FM3X,branch_code_K6QO,branch_code_QBOL,branch_code_JWFN,branch_code_JZ9D,branch_code_J9JW,branch_code_GHYX,branch_code_ECY3,sex_P5DA,sex_RIBP,sex_8NN1,sex_7POT,sex_66FJ,sex_GYSR,sex_SOP4,sex_RVSZ,sex_PYUQ,sex_LJR9,sex_N2MW,sex_AHXO,sex_BSTQ,sex_FM3X,sex_K6QO,sex_QBOL,sex_JWFN,sex_JZ9D,sex_J9JW,sex_GHYX,sex_ECY3
sex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
F,17421899,17,351,48,96,71,0,77,7463,689,114,238,146,88,31,6540,2106,102,585,580,406,259,9.770434,505.217082,49.325665,99.170339,95.302932,0.99556,122.645411,7624.964554,695.694042,111.440856,238.181475,150.538355,100.648451,34.835382,6510.368698,2168.963426,100.991234,441.383331,438.895284,280.333691,313.768719,17.0,351.0,48.0,96.0,71.0,0.0,77.0,7463.0,689.0,114.0,238.0,146.0,88.0,31.0,6540.0,2106.0,102.0,585.0,580.0,406.0,259.0
M,40245392,23,1429,109,220,268,4,354,17865,1484,240,600,393,236,79,15089,4727,209,840,838,496,843,30.229566,1274.782918,107.674335,216.829661,243.697068,3.00444,308.354589,17703.035446,1477.305958,242.559144,599.818525,388.461645,223.351549,75.164618,15118.631302,4664.036574,210.008766,983.616669,979.104716,621.666309,788.231281,23.0,1429.0,109.0,220.0,268.0,4.0,354.0,17865.0,1484.0,240.0,600.0,393.0,236.0,79.0,15089.0,4727.0,209.0,840.0,838.0,496.0,843.0


In [256]:
train.groupby('birth_year').sum()
#Seems like RVSZ, K6QO and QBOL are three most popular policies. 
#On the other hand, GYSR(only 4 people took it) and P5DA are the worst. FM3X, 8NN1 and JWFN are the bad.

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,branch_code_P5DA,branch_code_RIBP,branch_code_8NN1,branch_code_7POT,branch_code_66FJ,branch_code_GYSR,branch_code_SOP4,branch_code_RVSZ,branch_code_PYUQ,branch_code_LJR9,branch_code_N2MW,branch_code_AHXO,branch_code_BSTQ,branch_code_FM3X,branch_code_K6QO,branch_code_QBOL,branch_code_JWFN,branch_code_JZ9D,branch_code_J9JW,branch_code_GHYX,branch_code_ECY3,sex_P5DA,sex_RIBP,sex_8NN1,sex_7POT,sex_66FJ,sex_GYSR,sex_SOP4,sex_RVSZ,sex_PYUQ,sex_LJR9,sex_N2MW,sex_AHXO,sex_BSTQ,sex_FM3X,sex_K6QO,sex_QBOL,sex_JWFN,sex_JZ9D,sex_J9JW,sex_GHYX,sex_ECY3
birth_year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1932,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.001570,0.073783,0.016013,0.031083,0.009105,0.000000,0.014443,0.796546,0.171115,0.030455,0.012559,0.001256,0.001256,0.001884,0.599058,0.424804,0.003454,0.028885,0.028571,0.059341,0.020094,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468
1936,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001570,0.073783,0.016013,0.031083,0.009105,0.000000,0.014443,0.796546,0.171115,0.030455,0.012559,0.001256,0.001256,0.001884,0.599058,0.424804,0.003454,0.028885,0.028571,0.059341,0.020094,0.001931,0.039873,0.005453,0.010905,0.008065,0.000000,0.008747,0.847779,0.078269,0.012950,0.027036,0.016585,0.009997,0.003522,0.742929,0.239237,0.011587,0.066455,0.065887,0.046121,0.029422
1938,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001570,0.073783,0.016013,0.031083,0.009105,0.000000,0.014443,0.796546,0.171115,0.030455,0.012559,0.001256,0.001256,0.001884,0.599058,0.424804,0.003454,0.028885,0.028571,0.059341,0.020094,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468
1946,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000369,0.143314,0.009710,0.017822,0.026794,0.000123,0.030605,0.887045,0.066740,0.017085,0.010447,0.001967,0.014503,0.005162,0.716445,0.238569,0.003073,0.024705,0.024705,0.031957,0.027040,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,0.878794,0.072999,0.011806,0.029514,0.019332,0.011609,0.003886,0.742240,0.232525,0.010281,0.041320,0.041222,0.024399,0.041468
1947,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.000737,0.286627,0.019420,0.035644,0.053589,0.000246,0.061209,1.774090,0.133481,0.034169,0.020895,0.003933,0.029007,0.010324,1.432891,0.477139,0.006146,0.049410,0.049410,0.063913,0.054081,0.002263,0.140587,0.010724,0.021644,0.026366,0.000394,0.034827,1.757588,0.145998,0.023612,0.059029,0.038664,0.023218,0.007772,1.484480,0.465050,0.020562,0.082641,0.082444,0.048797,0.082936
1948,1,2,2,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0.262515,0.586296,0.047535,0.089309,0.101337,0.020325,0.070080,2.483474,0.359799,0.069352,1.063118,0.797542,0.035961,0.007046,1.738721,0.961280,0.048541,0.085504,0.085190,0.107840,0.100421,0.006125,0.220333,0.021629,0.043455,0.042497,0.000394,0.052321,3.453146,0.302536,0.049512,0.113101,0.071834,0.043211,0.014815,2.970337,0.943523,0.043736,0.215550,0.214217,0.141039,0.141779
1950,0,3,1,0,3,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0.002676,0.536523,0.049064,0.093818,0.097332,0.000369,0.118022,4.387094,0.416256,0.094900,0.052101,0.008938,0.047618,0.018797,3.588323,1.385788,0.060800,0.120469,0.119086,0.189792,0.126171,0.005657,0.351468,0.026809,0.054110,0.065916,0.000984,0.087068,4.393969,0.364996,0.059029,0.147572,0.096660,0.058045,0.019430,3.711201,1.162625,0.051404,0.206601,0.206109,0.121993,0.207339
1951,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.001570,0.073783,0.016013,0.031083,0.009105,0.000000,0.014443,0.796546,0.171115,0.030455,0.012559,0.001256,0.001256,0.001884,0.599058,0.424804,0.003454,0.028885,0.028571,0.059341,0.020094,0.001131,0.070294,0.005362,0.010822,0.013183,0.000197,0.017414,

In [257]:
pd.value_counts(train.occupation_code)

2A7I    7089
0KID    3969
SST3    2711
UJ5T    2079
0OJM    1565
BPSA    1091
QZYX     856
31JW     683
0B60     566
BP09     525
RUFT     456
6PE7     448
6E4H     438
U37O     402
5FPK     347
3NHZ     332
J9SY     323
N2ZZ     261
YMGT     248
OPVX     240
WMTK     235
B8W8     225
PMAI     217
4W0D     162
7UDQ     152
FSXG     124
P4MD     124
APO0     117
I31I     116
K5LB     112
1NFK     111
F57O     102
73AC     100
XC1N      99
S9KU      96
1MSV      88
Q57T      72
VREH      72
U9RX      69
T6AB      65
NQW1      61
9HD1      60
IQFS      59
E39I      57
SS6D      56
7G9M      55
WE7U      50
C8F6      47
JS7M      40
JI64      40
2BE6      39
WV7U      39
DPRV      34
SF1X      33
DZRV      33
31GG      32
00MO      31
Y1WG      30
7UHW      30
CV2C      30
XHJD      29
TUN1      29
44SU      28
A793      28
GWEP      27
C1E3      27
UBBX      25
H1K7      24
L4PL      24
R44Q      24
RY9B      24
OYQF      23
SSTX      22
2HLT      22
5JRZ      22
PKW3      22
XVMH      21

In [258]:
pd.value_counts(test.occupation_code)

2A7I    2405
0KID    1465
SST3     943
UJ5T     697
0OJM     542
BPSA     397
QZYX     302
31JW     204
0B60     187
BP09     183
RUFT     173
U37O     165
6E4H     134
6PE7     129
J9SY     107
3NHZ     106
5FPK     100
N2ZZ      84
WMTK      83
YMGT      79
B8W8      78
OPVX      76
PMAI      61
FSXG      50
1NFK      49
P4MD      46
7UDQ      46
4W0D      40
73AC      40
I31I      40
K5LB      39
APO0      37
S9KU      36
XC1N      36
F57O      30
T6AB      27
U9RX      25
IQFS      25
Q57T      23
1MSV      23
VREH      23
2BE6      21
9HD1      19
SS6D      19
E39I      18
7G9M      16
NQW1      14
00MO      14
C8F6      14
31GG      14
DZRV      12
JS7M      12
H1K7      12
WSID      11
0S50      11
GWEP      10
WE7U      10
7UHW      10
WV7U      10
C1E3      10
0PO7       9
FSWO       9
JI64       9
CV2C       9
DPRV       9
UBBX       9
LQ0W       9
A793       9
BWBW       8
1H8Y       8
RXV3       8
SSTX       8
ZKQ3       8
RY9B       8
TUN1       7
5JRZ       7
XVMH       7

In [259]:
set1 = set(train.occupation_code.unique()) # Checking differences in branch_codes between test/train sets.
set2 = set(test.occupation_code.unique())
np.setdiff1d(set1, set2) # Checked both ways. Some in train but not test, vice versa

array([{'HTQS', 'S96O', 'HSVE', 'IE90', 'V4XX', 'SST3', '1AN5', 'UBBX', '7UHW', 'PKW3', '2HLT', 'FSXG', 'IUT9', 'SF1X', '374O', 'S9KU', '0SH6', 'BER4', 'RXV3', 'URYD', 'OEH6', '0B60', '8HRZ', 'E5PF', '5JRZ', '1YKL', 'BWBW', '9IP9', 'YX47', '0KID', '8Y24', 'IQFS', 'GQ0N', 'P4MD', '9HD1', 'W1X2', 'HAXM', '2R78', 'L4PL', 'RSN9', 'GWEP', '2G86', 'L1P3', 'IX8T', '6XXU', 'VREH', '0S50', 'AQIB', 'H1K7', '44SU', 'W3Y9', 'AIIN', 'QZYX', '820B', 'LGTN', 'FSWO', '5OVC', 'F35Z', '1NFK', 'R44Q', '9B5B', 'CV2C', 'K5LB', 'WE7U', '2MBB', 'N2ZZ', 'PMAI', '1H8Y', 'DD8W', 'KNVN', 'CEL6', 'CYDC', 'W3ZV', 'FLNZ', 'QQVA', 'RE69', '734F', 'DZRV', 'MEFQ', '7KM4', 'J9SY', 'PWCW', '9IM8', 'WSID', 'C8F6', 'JQH3', '738L', '7UDQ', 'OYQF', 'NQW1', 'Q2L0', 'RH2K', 'WMTK', 'Q231', 'LLLH', '1DT6', 'JHU5', '0OJM', 'NSJX', 'OQMY', 'JS7M', 'LAYD', '2XZ1', 'ZHC2', 'ZWPL', '6KYM', '0VYC', '2346', 'M0WG', 'QQUP', 'VYSA', '4M0E', 'BP09', 'BX9E', '2JHV', '1MB4', 'CAAV', 'FLXH', 'JSAX', '73AC', 'OPVX', 'RM3L', 'YMGT', 'XVMH', 

In [260]:
#New categorys in occupation_code in test replace to most popular in train
replace_new_values = lambda s: '2A7I' if s not in [
'2A7I',
'0KID',
'SST3',
'UJ5T',
'0OJM',
'BPSA',
'QZYX',
'31JW',
'0B60',
'BP09',
'RUFT',
'6PE7',
'6E4H',
'U37O',
'5FPK',
'3NHZ',
'J9SY',
'N2ZZ',
'YMGT',
'OPVX',
'WMTK',
'B8W8',
'PMAI',
'4W0D',
'7UDQ',
'FSXG',
'P4MD',
'APO0',
'I31I',
'K5LB',
'1NFK',
'F57O',
'73AC',
'XC1N',
'S9KU',
'1MSV',
'Q57T',
'VREH',
'U9RX',
'T6AB',
'NQW1',
'9HD1',
'IQFS',
'E39I',
'SS6D',
'7G9M',
'WE7U',
'C8F6',
'JS7M',
'JI64',
'WV7U',
'2BE6',
'DPRV',
'SF1X',
'DZRV',
'31GG',
'00MO',
'CV2C',
'Y1WG',
'7UHW',
'XHJD',
'TUN1',
'44SU',
'A793',
'GWEP',
'C1E3',
'UBBX',
'R44Q',
'L4PL',
'H1K7',
'RY9B',
'OYQF',
'2HLT',
'5JRZ',
'PKW3',
'SSTX',
'XVMH',
'2JHV',
'LQ0W',
'2YAO',
'K5GV',
'BWBW',
'FSWO',
'NSJX',
'NO3L',
'JHU5',
'AQIB',
'Q6J6',
'0PO7',
'RE69',
'F35Z',
'RXV3',
'WSID',
'0S50',
'WE0G',
'L1P3',
'ZCQR',
'1H8Y',
'ZKQ3',
'GVZ1',
'JN20',
'B3QW',
'QX54',
'FLNZ',
'9B5B',
'QJID',
'NFJH',
'734F',
'8HRZ',
'KPG9',
'IUT9',
'FJBW',
'9IM8',
'9IP9',
'3X46',
'K0DL',
'7KM4',
'74BF',
'LAYD',
'WVQF',
'QQUP',
'MU16',
'820B',
'AIIN',
'6YZA',
'6KYM',
'LGTN',
'Q2L0',
'GQ0N',
'RF6M',
'6XXU',
'834U',
'V4XX',
'INEJ',
'1MB4',
'WIWP',
'W1X2',
'DHSN',
'374O',
'IJ01',
'PSUY',
'2XZ1',
'RSN9',
'2346',
'S96O',
'8Y24',
'0SH6',
'2G86',
'IMHI',
'JUIP',
'ZWPL',
'RM3L',
'2MBB',
'0VYC',
'KNVN',
'Q0LY',
'Y7G1',
'IZ77',
'7UWC',
'9FA1',
'UC7E',
'HJF4',
'N7K2',
'2686',
'PWCW',
'QS0L',
'EE5R',
'E5PF',
'2US6',
'IE90',
'DE5D',
'ONY7',
'Z7PM',
'HTQS',
'LLLH',
'RH2K',
'HAXM',
'3YQ1',
'ZA1S',
'X1JO',
'W3Y9',
'YX47',
'1YKL',
'VVTC',
'GZA8',
'AIDS',
'6LKA',
'NX5Y',
'QQVA',
'JQH3',
'NDL9',
'Q231',
'FLXH',
'738L',
'UYDZ',
'KBWO',
'6SKY',
'A4ZC',
'OQMY',
'1AN5',
'W3ZV',
'OEH6',
'OME4',
'ZHC2',
'PJR4',
'BER4',
'PPNK',
'BFD1',
'WSRG',
'CEL6',
'M0WG',
'URYD',
'1DT6',
'5OVC',
'4M0E',
'YJXM',
'CYDC',
'59QM',
'BX9E',
'JSAX',
'R7GL',
'I2OD',
'VZN9',
'P2K2',
'DD8W',
'IX8T',
'CAAV',
'5LNN',
'KUPK',
'VYSA',
'2R78',
'HSVE',
'MEFQ'

] else s
test['occupation_code'] = test['occupation_code'].map(replace_new_values)

In [261]:
#master
list_of_features = [ 'ID', 'branch_code_P5DA', 'branch_code_RIBP', 'branch_code_8NN1',
       'branch_code_7POT', 'branch_code_66FJ', 'branch_code_GYSR', 'branch_code_SOP4', 'branch_code_RVSZ', 
                   'branch_code_PYUQ', 'branch_code_LJR9', 'branch_code_N2MW', 'branch_code_AHXO',
                    'branch_code_BSTQ', 'branch_code_FM3X', 'branch_code_K6QO', 'branch_code_QBOL', 'branch_code_JWFN',
                   'branch_code_JZ9D', 'branch_code_J9JW', 'branch_code_GHYX', 'branch_code_ECY3' ]

extratr = train[list_of_features]
extratest = test[list_of_features]

list_of_features2  = [  'branch_code_P5DA', 'branch_code_RIBP', 'branch_code_8NN1',
       'branch_code_7POT', 'branch_code_66FJ', 'branch_code_GYSR', 'branch_code_SOP4', 'branch_code_RVSZ', 
                   'branch_code_PYUQ', 'branch_code_LJR9', 'branch_code_N2MW', 'branch_code_AHXO',
                    'branch_code_BSTQ', 'branch_code_FM3X', 'branch_code_K6QO', 'branch_code_QBOL', 'branch_code_JWFN',
                   'branch_code_JZ9D', 'branch_code_J9JW', 'branch_code_GHYX', 'branch_code_ECY3' ]

In [262]:
train.drop(list_of_features2, axis=1, inplace=True)
test.drop(list_of_features2, axis=1, inplace=True)


In [263]:
list_of_features1 = [ 'ID', 'sex_P5DA', 'sex_RIBP', 'sex_8NN1',
       'sex_7POT', 'sex_66FJ', 'sex_GYSR', 'sex_SOP4', 'sex_RVSZ', 
                   'sex_PYUQ', 'sex_LJR9', 'sex_N2MW', 'sex_AHXO',
                    'sex_BSTQ', 'sex_FM3X', 'sex_K6QO', 'sex_QBOL', 'sex_JWFN',
                   'sex_JZ9D', 'sex_J9JW', 'sex_GHYX', 'sex_ECY3' ]

extratr1 = train[list_of_features1]
extratest1 = test[list_of_features1]

list_of_features3  = [  'sex_P5DA', 'sex_RIBP', 'sex_8NN1',
       'sex_7POT', 'sex_66FJ', 'sex_GYSR', 'sex_SOP4', 'sex_RVSZ', 
                   'sex_PYUQ', 'sex_LJR9', 'sex_N2MW', 'sex_AHXO',
                    'sex_BSTQ', 'sex_FM3X', 'sex_K6QO', 'sex_QBOL', 'sex_JWFN',
                   'sex_JZ9D', 'sex_J9JW', 'sex_GHYX', 'sex_ECY3' ]


In [264]:
train.drop(list_of_features3, axis=1, inplace=True)
test.drop(list_of_features3, axis=1, inplace=True)

### New format

In [265]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for i in index:
    c+=1
    for k in range(len(binary)):
      if k == i:
        binary_transformed = list(copy.copy(binary))
        binary_transformed[i] = 0
        X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])

X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2' ]

In [266]:
X_test = []
true_values = []
c = 0
for v in test.values:
  c += 1
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  X_test.append(list(info) + list(binary) + [c])
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']  
                                    
                  
                 
                 
                  
                 

### Transform data

In [267]:
features_train = []
features_test = []
columns = []

append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
                  
for v in append_features:
  features_train.append(X_train[v].values.reshape(-1, 1))
  features_test.append(X_test[v].values.reshape(-1, 1))
  columns.append(np.array([v]))

y_train = X_train[['product_pred']]

In [268]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))

X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

### New features

In [269]:
X_train['join_date'] = pd.to_datetime(X_train['join_date'])
# выделяем из переменной data_start ГОДА
X_train['year'] = pd.DatetimeIndex(X_train['join_date']).year
# а еще можно так data['year'] = data['date_start'].dt.year
# выделяем из переменной data_start КВАРТАЛЫ
X_train['quarter'] = pd.DatetimeIndex(X_train['join_date']).quarter
# а еще можно так data['quarter'] = data['date_start'].dt.quarter
# выделяем из переменной data_start МЕСЯЦЫ, где 1 - январь, 12 - декабрь
X_train['month'] = pd.DatetimeIndex(X_train['join_date']).month
# а еще можно так data['month'] = data['date_start'].dt.month
# выделяем из переменной data_start ПОРЯДКОВЫЕ НОМЕРА ДНЕЙ от 1 до 31
X_train['day'] = pd.DatetimeIndex(X_train['join_date']).day
# а еще можно так data['day'] = data['date_start'].dt.day
# выделяем из переменной data_start ДНИ НЕДЕЛИ от 0 до 6, где 0 - понедельник, 6 - воскресенье
X_train['dayofweek'] = pd.DatetimeIndex(X_train['join_date']).dayofweek
# а еще можно так data['dayofweek'] = data['date_start'].dt.dayofweek
# выделяем из переменной data_start НАЗВАНИЯ ДНЕЙ НЕДЕЛИ
#X_train['dayofyear'] = pd.DatetimeIndex(X_train['join_date']).dayofyear
# а еще можно так data['weekday_name'] = data['date_start'].dt.weekday_name
#X_train['month_diff'] = ((datetime.datetime.today() - X_train['join_date']).dt.days)//30
#X_train['is_month_start'] = X_train['join_date'].dt.is_month_start
#X_train['weekofyear'] = X_train['join_date'].dt.weekofyear
#X_train['weekend'] = (X_train.join_date.dt.weekday >=5).astype(int)
X_train.drop('join_date', axis=1, inplace=True)


X_test['join_date'] = pd.to_datetime(X_test['join_date'])
# выделяем из переменной data_start ГОДА
X_test['year'] = pd.DatetimeIndex(X_test['join_date']).year
# а еще можно так data['year'] = data['date_start'].dt.year
# выделяем из переменной data_start КВАРТАЛЫ
X_test['quarter'] = pd.DatetimeIndex(X_test['join_date']).quarter
# а еще можно так data['quarter'] = data['date_start'].dt.quarter
# выделяем из переменной data_start МЕСЯЦЫ, где 1 - январь, 12 - декабрь
X_test['month'] = pd.DatetimeIndex(X_test['join_date']).month
# а еще можно так data['month'] = data['date_start'].dt.month
# выделяем из переменной data_start ПОРЯДКОВЫЕ НОМЕРА ДНЕЙ от 1 до 31
X_test['day'] = pd.DatetimeIndex(X_test['join_date']).day
# а еще можно так data['day'] = data['date_start'].dt.day
# выделяем из переменной data_start ДНИ НЕДЕЛИ от 0 до 6, где 0 - понедельник, 6 - воскресенье
X_test['dayofweek'] = pd.DatetimeIndex(X_test['join_date']).dayofweek
# а еще можно так data['dayofweek'] = data['date_start'].dt.dayofweek
# выделяем из переменной data_start НАЗВАНИЯ ДНЕЙ НЕДЕЛИ
#X_test['dayofyear'] = pd.DatetimeIndex(X_test['join_date']).dayofyear
# а еще можно так data['weekday_name'] = data['date_start'].dt.weekday_name
#X_test['is_month_start'] = X_test['join_date'].dt.is_month_start
#X_test['weekofyear'] = X_test['join_date'].dt.weekofyear
#X_test['month_diff'] = ((datetime.datetime.today() - X_test['join_date']).dt.days)//30
#X_test['weekend'] = (X_test.join_date.dt.weekday >=5).astype(int)
X_test.drop('join_date', axis=1, inplace=True)




X_train['date_diff'] = X_train['year'] - X_train['birth_year']
X_test['date_diff'] = X_test['year'] - X_test['birth_year']
X_train['date_diff'] = X_train['date_diff'].astype('float')
X_test['date_diff'] = X_test['date_diff'].astype('float')







#X_train['branch_code + occupation_code'] = X_train.apply(lambda x: f"{x['branch_code']} + {x['occupation_code']}", axis=1)
#X_test['branch_code + occupation_code'] = X_test.apply(lambda x: f"{x['branch_code']} + {x['occupation_code']}", axis=1)
#
#X_train['branch_code + occupation_category_code'] = X_train.apply(lambda x: f"{x['branch_code']} + {x['occupation_category_code']}", axis=1)
#X_test['branch_code + occupation_category_code'] = X_test.apply(lambda x: f"{x['branch_code']} + {x['occupation_category_code']}", axis=1)                                                        

#X_train['occupation_code+occupation_category_code'] = X_train.apply(lambda x: f"{x['occupation_code']} + {x['occupation_category_code']}", axis=1)
#X_test['occupation_code+occupation_category_code'] = X_test.apply(lambda x: f"{x['occupation_code']} + {x['occupation_category_code']}", axis=1)
                                                        






In [270]:
X_train.shape, X_test.shape

((66353, 35), (10000, 35))

In [271]:
X_train = X_train.merge(extratr, how='left', on='ID')
X_test = X_test.merge(extratest, how='left', on='ID')

In [272]:
X_train = X_train.merge(extratr1, how='left', on='ID')
X_test = X_test.merge(extratest1, how='left', on='ID')

In [273]:
print(X_train.isnull().sum())

P5DA                        0
RIBP                        0
8NN1                        0
7POT                        0
66FJ                        0
GYSR                        0
SOP4                        0
RVSZ                        0
PYUQ                        0
LJR9                        0
N2MW                        0
AHXO                        0
BSTQ                        0
FM3X                        0
K6QO                        0
QBOL                        0
JWFN                        0
JZ9D                        0
J9JW                        0
GHYX                        0
ECY3                        0
ID                          0
ID2                         0
sex                         0
marital_status              0
branch_code                 0
occupation_code             0
occupation_category_code    0
birth_year                  0
year                        7
quarter                     7
month                       7
day                         7
dayofweek 

In [274]:
print(X_test.isnull().sum())

P5DA                        0
RIBP                        0
8NN1                        0
7POT                        0
66FJ                        0
GYSR                        0
SOP4                        0
RVSZ                        0
PYUQ                        0
LJR9                        0
N2MW                        0
AHXO                        0
BSTQ                        0
FM3X                        0
K6QO                        0
QBOL                        0
JWFN                        0
JZ9D                        0
J9JW                        0
GHYX                        0
ECY3                        0
ID                          0
ID2                         0
sex                         0
marital_status              0
branch_code                 0
occupation_code             0
occupation_category_code    0
birth_year                  0
year                        1
quarter                     1
month                       1
day                         1
dayofweek 

### Cat encoding

In [275]:
X_train['year'].fillna(X_train['year'].median(), inplace=True)
X_train['quarter'].fillna(X_train['quarter'].median(), inplace=True)
X_train['month'].fillna(X_train['month'].median(), inplace=True)
X_train['day'].fillna(X_train['day'].median(), inplace=True)
X_train['dayofweek'].fillna(X_train['dayofweek'].median(), inplace=True)
X_train['date_diff'].fillna(X_train['date_diff'].median(), inplace=True)
#X_train['month_diff'].fillna(X_train['month_diff'].median(), inplace=True)
#X_train['weekofyear'].fillna(X_train['weekofyear'].median(), inplace=True)

 

X_test['year'].fillna(X_train['year'].median(), inplace=True)
X_test['quarter'].fillna(X_train['quarter'].median(), inplace=True)
X_test['month'].fillna(X_train['month'].median(), inplace=True)
X_test['day'].fillna(X_train['day'].median(), inplace=True)
X_test['dayofweek'].fillna(X_train['dayofweek'].median(), inplace=True)
X_test['date_diff'].fillna(X_train['date_diff'].median(), inplace=True)
#X_test['month_diff'].fillna(X_train['month_diff'].median(), inplace=True)
#X_test['weekofyear'].fillna(X_train['weekofyear'].median(), inplace=True)


In [276]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(le.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [277]:
abs_freq = X_train['sex'].value_counts()
X_train['sex_abs_freq'] = X_train['sex'].map(abs_freq)
X_test['sex_abs_freq'] = X_test['sex'].map(abs_freq)

abs_freq = X_train['marital_status'].value_counts()
X_train['marital_status_abs_freq'] = X_train['marital_status'].map(abs_freq)
X_test['marital_status_abs_freq'] = X_test['marital_status'].map(abs_freq)

abs_freq = X_train['branch_code'].value_counts()
X_train['branch_code_abs_freq'] = X_train['branch_code'].map(abs_freq)
X_test['branch_code_abs_freq'] = X_test['branch_code'].map(abs_freq)

abs_freq = X_train['occupation_code'].value_counts()
X_train['occupation_code_abs_freq'] = X_train['occupation_code'].map(abs_freq)
X_test['occupation_code_abs_freq'] = X_test['occupation_code'].map(abs_freq)

abs_freq = X_train['occupation_category_code'].value_counts()
X_train['occupation_category_code_abs_freq'] = X_train['occupation_category_code'].map(abs_freq)
X_test['occupation_category_code_abs_freq'] = X_test['occupation_category_code'].map(abs_freq)




cat = ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code']
X_train.drop(cat, axis=1, inplace=True)
X_test.drop(cat, axis=1, inplace=True)




In [278]:
listtype = ['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 
                   'PYUQ', 'LJR9', 'N2MW', 'AHXO',
                    'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN',
                   'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'birth_year'  ]
X_train[listtype ] = X_train[listtype].astype(int)
X_test[listtype ] = X_test[listtype].astype(int)

In [279]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66353 entries, 0 to 66352
Data columns (total 77 columns):
P5DA                                 66353 non-null int32
RIBP                                 66353 non-null int32
8NN1                                 66353 non-null int32
7POT                                 66353 non-null int32
66FJ                                 66353 non-null int32
GYSR                                 66353 non-null int32
SOP4                                 66353 non-null int32
RVSZ                                 66353 non-null int32
PYUQ                                 66353 non-null int32
LJR9                                 66353 non-null int32
N2MW                                 66353 non-null int32
AHXO                                 66353 non-null int32
BSTQ                                 66353 non-null int32
FM3X                                 66353 non-null int32
K6QO                                 66353 non-null int32
QBOL                 

# Ensembling model with spline calibration

In [237]:
#EnsembleVoteClassifier

X = X_train.drop(columns=['ID','ID2'], axis=1)
y = y_train
from mlxtend.classifier import EnsembleVoteClassifier
import warnings
import sklearn
import lightgbm as lgb
from xgboost import XGBClassifier, plot_importance

from catboost import CatBoostClassifier
warnings.filterwarnings('ignore')

# Create boosting classifiers
lgbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='multiclass', 
                                learning_rate=0.017, num_iterations=950, #950best 0.017
                                bagging_fraction=0.85, feature_fraction=0.9, reg_lambda=1.3, reg_alpha=1.91,num_class= 21,  
                                bagging_freq = 9, colsample_bytree= 0.75, n_jobs = -1,  seed = 236)

cat = CatBoostClassifier(n_estimators=1700,  task_type="GPU", learning_rate=0.022, depth=7, random_state=236, #learning_rate=0.022  n_estimators=1600 1700 best depth=7
                        cat_features = ['sex_abs_freq', 'marital_status_abs_freq', 'branch_code_abs_freq', 
                                       'occupation_code_abs_freq', 'occupation_category_code_abs_freq'])
                        
xgb_boost = XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1700, learning_rate=0.02, #max_depth=5 learning_rate=0.02  n_estimators=1500 1700best 1650 81submission
                        num_classes=21, tree_method = 'gpu_hist', seed = 236)

boost_array = [lgbm, cat, xgb_boost]

eclf = EnsembleVoteClassifier(clfs=[lgbm, cat, xgb_boost], 
                              voting='soft' #soft
                             )

#labels = ['lgbm', 'cat', 'xgb_boost', 'Ensemble']

#for clf, label in zip([lgbm, cat, xgb_boost , eclf], labels):
 #   scores = sklearn.model_selection.cross_val_score(clf, X, y, cv=5, scoring='accuracy') # f1_weighted scoring = 'average_precision' scoring_micro = make_scorer(precision_score, average='micro')
 #   print("Mean: {0:.3f}, std: (+/-) {1:.3f} [{2}]".format(scores.mean(), scores.std(), label))

In [238]:
proba  = eclf.fit(X, y).predict_proba(X_test.drop(columns=['ID','ID2'], axis=1))


0:	learn: 2.7366599	total: 85.3ms	remaining: 2m 24s
1:	learn: 2.5306629	total: 161ms	remaining: 2m 16s
2:	learn: 2.3716938	total: 226ms	remaining: 2m 7s
3:	learn: 2.2434618	total: 294ms	remaining: 2m 4s
4:	learn: 2.1340006	total: 357ms	remaining: 2m 1s
5:	learn: 2.0395934	total: 420ms	remaining: 1m 58s
6:	learn: 1.9568350	total: 489ms	remaining: 1m 58s
7:	learn: 1.8832315	total: 562ms	remaining: 1m 58s
8:	learn: 1.8170577	total: 633ms	remaining: 1m 58s
9:	learn: 1.7566128	total: 711ms	remaining: 2m
10:	learn: 1.7011752	total: 784ms	remaining: 2m
11:	learn: 1.6506143	total: 866ms	remaining: 2m 1s
12:	learn: 1.6040175	total: 935ms	remaining: 2m 1s
13:	learn: 1.5605637	total: 1s	remaining: 2m
14:	learn: 1.5200522	total: 1.07s	remaining: 2m
15:	learn: 1.4822006	total: 1.14s	remaining: 2m
16:	learn: 1.4464906	total: 1.2s	remaining: 1m 58s
17:	learn: 1.4127221	total: 1.26s	remaining: 1m 57s
18:	learn: 1.3811945	total: 1.33s	remaining: 1m 57s
19:	learn: 1.3508883	total: 1.39s	remaining: 1m 56

158:	learn: 0.5051414	total: 11s	remaining: 1m 46s
159:	learn: 0.5043995	total: 11.1s	remaining: 1m 46s
160:	learn: 0.5034685	total: 11.1s	remaining: 1m 46s
161:	learn: 0.5026289	total: 11.2s	remaining: 1m 46s
162:	learn: 0.5018115	total: 11.2s	remaining: 1m 46s
163:	learn: 0.5009610	total: 11.3s	remaining: 1m 45s
164:	learn: 0.5000643	total: 11.4s	remaining: 1m 45s
165:	learn: 0.4991453	total: 11.5s	remaining: 1m 45s
166:	learn: 0.4982530	total: 11.5s	remaining: 1m 45s
167:	learn: 0.4973722	total: 11.6s	remaining: 1m 45s
168:	learn: 0.4964683	total: 11.7s	remaining: 1m 45s
169:	learn: 0.4955222	total: 11.7s	remaining: 1m 45s
170:	learn: 0.4946442	total: 11.8s	remaining: 1m 45s
171:	learn: 0.4939962	total: 11.8s	remaining: 1m 45s
172:	learn: 0.4932123	total: 11.9s	remaining: 1m 45s
173:	learn: 0.4923443	total: 12s	remaining: 1m 45s
174:	learn: 0.4915556	total: 12s	remaining: 1m 44s
175:	learn: 0.4908239	total: 12.1s	remaining: 1m 44s
176:	learn: 0.4899648	total: 12.2s	remaining: 1m 44s

314:	learn: 0.4269160	total: 21.5s	remaining: 1m 34s
315:	learn: 0.4265998	total: 21.5s	remaining: 1m 34s
316:	learn: 0.4263389	total: 21.6s	remaining: 1m 34s
317:	learn: 0.4260898	total: 21.6s	remaining: 1m 34s
318:	learn: 0.4257768	total: 21.7s	remaining: 1m 33s
319:	learn: 0.4254942	total: 21.8s	remaining: 1m 33s
320:	learn: 0.4251973	total: 21.8s	remaining: 1m 33s
321:	learn: 0.4248357	total: 21.9s	remaining: 1m 33s
322:	learn: 0.4245501	total: 22s	remaining: 1m 33s
323:	learn: 0.4242443	total: 22s	remaining: 1m 33s
324:	learn: 0.4240354	total: 22.1s	remaining: 1m 33s
325:	learn: 0.4237617	total: 22.2s	remaining: 1m 33s
326:	learn: 0.4235484	total: 22.2s	remaining: 1m 33s
327:	learn: 0.4232646	total: 22.3s	remaining: 1m 33s
328:	learn: 0.4230562	total: 22.4s	remaining: 1m 33s
329:	learn: 0.4226522	total: 22.4s	remaining: 1m 33s
330:	learn: 0.4222673	total: 22.5s	remaining: 1m 33s
331:	learn: 0.4219700	total: 22.6s	remaining: 1m 33s
332:	learn: 0.4217522	total: 22.7s	remaining: 1m 3

470:	learn: 0.3919706	total: 31.9s	remaining: 1m 23s
471:	learn: 0.3918451	total: 31.9s	remaining: 1m 23s
472:	learn: 0.3917343	total: 32s	remaining: 1m 23s
473:	learn: 0.3915320	total: 32.1s	remaining: 1m 22s
474:	learn: 0.3913448	total: 32.2s	remaining: 1m 22s
475:	learn: 0.3911054	total: 32.2s	remaining: 1m 22s
476:	learn: 0.3909874	total: 32.3s	remaining: 1m 22s
477:	learn: 0.3908271	total: 32.4s	remaining: 1m 22s
478:	learn: 0.3906298	total: 32.4s	remaining: 1m 22s
479:	learn: 0.3903461	total: 32.5s	remaining: 1m 22s
480:	learn: 0.3902114	total: 32.6s	remaining: 1m 22s
481:	learn: 0.3900877	total: 32.6s	remaining: 1m 22s
482:	learn: 0.3899855	total: 32.7s	remaining: 1m 22s
483:	learn: 0.3898020	total: 32.7s	remaining: 1m 22s
484:	learn: 0.3896364	total: 32.8s	remaining: 1m 22s
485:	learn: 0.3894481	total: 32.9s	remaining: 1m 22s
486:	learn: 0.3892872	total: 32.9s	remaining: 1m 22s
487:	learn: 0.3891201	total: 33s	remaining: 1m 21s
488:	learn: 0.3889843	total: 33.1s	remaining: 1m 2

626:	learn: 0.3691779	total: 42.3s	remaining: 1m 12s
627:	learn: 0.3690377	total: 42.4s	remaining: 1m 12s
628:	learn: 0.3688986	total: 42.5s	remaining: 1m 12s
629:	learn: 0.3688038	total: 42.5s	remaining: 1m 12s
630:	learn: 0.3686121	total: 42.6s	remaining: 1m 12s
631:	learn: 0.3684304	total: 42.7s	remaining: 1m 12s
632:	learn: 0.3682797	total: 42.7s	remaining: 1m 12s
633:	learn: 0.3681809	total: 42.8s	remaining: 1m 11s
634:	learn: 0.3680566	total: 42.9s	remaining: 1m 11s
635:	learn: 0.3679732	total: 42.9s	remaining: 1m 11s
636:	learn: 0.3678945	total: 43s	remaining: 1m 11s
637:	learn: 0.3677917	total: 43.1s	remaining: 1m 11s
638:	learn: 0.3676843	total: 43.1s	remaining: 1m 11s
639:	learn: 0.3676026	total: 43.2s	remaining: 1m 11s
640:	learn: 0.3674915	total: 43.3s	remaining: 1m 11s
641:	learn: 0.3673847	total: 43.3s	remaining: 1m 11s
642:	learn: 0.3672951	total: 43.4s	remaining: 1m 11s
643:	learn: 0.3672050	total: 43.5s	remaining: 1m 11s
644:	learn: 0.3671063	total: 43.5s	remaining: 1m

...

945:	learn: 0.3411775	total: 1m 3s	remaining: 50.2s
946:	learn: 0.3410755	total: 1m 3s	remaining: 50.1s
947:	learn: 0.3410004	total: 1m 3s	remaining: 50.1s
948:	learn: 0.3409614	total: 1m 3s	remaining: 50s
949:	learn: 0.3408811	total: 1m 3s	remaining: 49.9s
950:	learn: 0.3408484	total: 1m 3s	remaining: 49.9s
951:	learn: 0.3407617	total: 1m 3s	remaining: 49.8s
952:	learn: 0.3407148	total: 1m 3s	remaining: 49.7s
953:	learn: 0.3406819	total: 1m 3s	remaining: 49.7s
954:	learn: 0.3406225	total: 1m 3s	remaining: 49.6s
955:	learn: 0.3405475	total: 1m 3s	remaining: 49.5s
956:	learn: 0.3404735	total: 1m 3s	remaining: 49.5s
957:	learn: 0.3404368	total: 1m 3s	remaining: 49.4s
958:	learn: 0.3403549	total: 1m 3s	remaining: 49.3s
959:	learn: 0.3403061	total: 1m 3s	remaining: 49.3s
960:	learn: 0.3402674	total: 1m 3s	remaining: 49.2s
961:	learn: 0.3402054	total: 1m 4s	remaining: 49.1s
962:	learn: 0.3401552	total: 1m 4s	remaining: 49s
963:	learn: 0.3400868	total: 1m 4s	remaining: 49s
964:	learn: 0.3400

1101:	learn: 0.3321333	total: 1m 12s	remaining: 39.4s
1102:	learn: 0.3320396	total: 1m 12s	remaining: 39.4s
1103:	learn: 0.3319854	total: 1m 12s	remaining: 39.3s
1104:	learn: 0.3319192	total: 1m 12s	remaining: 39.2s
1105:	learn: 0.3318586	total: 1m 12s	remaining: 39.2s
1106:	learn: 0.3318255	total: 1m 12s	remaining: 39.1s
1107:	learn: 0.3317686	total: 1m 13s	remaining: 39s
1108:	learn: 0.3316655	total: 1m 13s	remaining: 39s
1109:	learn: 0.3316108	total: 1m 13s	remaining: 38.9s
1110:	learn: 0.3315532	total: 1m 13s	remaining: 38.8s
1111:	learn: 0.3315087	total: 1m 13s	remaining: 38.8s
1112:	learn: 0.3314717	total: 1m 13s	remaining: 38.7s
1113:	learn: 0.3313657	total: 1m 13s	remaining: 38.6s
1114:	learn: 0.3313120	total: 1m 13s	remaining: 38.6s
1115:	learn: 0.3312692	total: 1m 13s	remaining: 38.5s
1116:	learn: 0.3312099	total: 1m 13s	remaining: 38.4s
1117:	learn: 0.3311532	total: 1m 13s	remaining: 38.4s
1118:	learn: 0.3310700	total: 1m 13s	remaining: 38.3s
1119:	learn: 0.3310142	total: 1m

1254:	learn: 0.3244418	total: 1m 22s	remaining: 29.1s
1255:	learn: 0.3243906	total: 1m 22s	remaining: 29.1s
1256:	learn: 0.3243736	total: 1m 22s	remaining: 29s
1257:	learn: 0.3243461	total: 1m 22s	remaining: 28.9s
1258:	learn: 0.3242872	total: 1m 22s	remaining: 28.9s
1259:	learn: 0.3242474	total: 1m 22s	remaining: 28.8s
1260:	learn: 0.3242067	total: 1m 22s	remaining: 28.7s
1261:	learn: 0.3241485	total: 1m 22s	remaining: 28.7s
1262:	learn: 0.3241246	total: 1m 22s	remaining: 28.6s
1263:	learn: 0.3241060	total: 1m 22s	remaining: 28.5s
1264:	learn: 0.3240840	total: 1m 22s	remaining: 28.5s
1265:	learn: 0.3240413	total: 1m 22s	remaining: 28.4s
1266:	learn: 0.3240096	total: 1m 22s	remaining: 28.4s
1267:	learn: 0.3239742	total: 1m 23s	remaining: 28.3s
1268:	learn: 0.3239367	total: 1m 23s	remaining: 28.2s
1269:	learn: 0.3238624	total: 1m 23s	remaining: 28.2s
1270:	learn: 0.3237877	total: 1m 23s	remaining: 28.1s
1271:	learn: 0.3237601	total: 1m 23s	remaining: 28s
1272:	learn: 0.3237137	total: 1m

1407:	learn: 0.3182837	total: 1m 31s	remaining: 19s
1408:	learn: 0.3182332	total: 1m 31s	remaining: 19s
1409:	learn: 0.3182231	total: 1m 31s	remaining: 18.9s
1410:	learn: 0.3182143	total: 1m 31s	remaining: 18.8s
1411:	learn: 0.3181962	total: 1m 31s	remaining: 18.8s
1412:	learn: 0.3181338	total: 1m 32s	remaining: 18.7s
1413:	learn: 0.3180749	total: 1m 32s	remaining: 18.6s
1414:	learn: 0.3180294	total: 1m 32s	remaining: 18.6s
1415:	learn: 0.3180143	total: 1m 32s	remaining: 18.5s
1416:	learn: 0.3179725	total: 1m 32s	remaining: 18.4s
1417:	learn: 0.3179217	total: 1m 32s	remaining: 18.4s
1418:	learn: 0.3178774	total: 1m 32s	remaining: 18.3s
1419:	learn: 0.3178307	total: 1m 32s	remaining: 18.2s
1420:	learn: 0.3177864	total: 1m 32s	remaining: 18.2s
1421:	learn: 0.3177408	total: 1m 32s	remaining: 18.1s
1422:	learn: 0.3177113	total: 1m 32s	remaining: 18s
1423:	learn: 0.3176942	total: 1m 32s	remaining: 18s
1424:	learn: 0.3176526	total: 1m 32s	remaining: 17.9s
1425:	learn: 0.3176168	total: 1m 32s

1560:	learn: 0.3122195	total: 1m 41s	remaining: 9s
1561:	learn: 0.3121943	total: 1m 41s	remaining: 8.94s
1562:	learn: 0.3121439	total: 1m 41s	remaining: 8.87s
1563:	learn: 0.3120765	total: 1m 41s	remaining: 8.81s
1564:	learn: 0.3120197	total: 1m 41s	remaining: 8.74s
1565:	learn: 0.3119817	total: 1m 41s	remaining: 8.68s
1566:	learn: 0.3119572	total: 1m 41s	remaining: 8.61s
1567:	learn: 0.3119194	total: 1m 41s	remaining: 8.54s
1568:	learn: 0.3118830	total: 1m 41s	remaining: 8.48s
1569:	learn: 0.3118753	total: 1m 41s	remaining: 8.41s
1570:	learn: 0.3118592	total: 1m 41s	remaining: 8.35s
1571:	learn: 0.3118342	total: 1m 41s	remaining: 8.28s
1572:	learn: 0.3117851	total: 1m 41s	remaining: 8.22s
1573:	learn: 0.3117726	total: 1m 41s	remaining: 8.15s
1574:	learn: 0.3117566	total: 1m 41s	remaining: 8.09s
1575:	learn: 0.3117082	total: 1m 41s	remaining: 8.02s
1576:	learn: 0.3116534	total: 1m 42s	remaining: 7.96s
1577:	learn: 0.3116176	total: 1m 42s	remaining: 7.89s
1578:	learn: 0.3115739	total: 1

# splinecalibration

In [239]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, log_loss, label_ranking_loss, label_ranking_average_precision_score, coverage_error
from sklearn.metrics import accuracy_score, confusion_matrix


import ml_insights as mli

mli.__version__

'0.1.4'

In [240]:
y = y.to_numpy()

In [241]:
#~1.5 hour calculation I7 7700 1060 gpu 16gb ram
cv_train_preds = mli.cv_predictions(eclf, X, y)

0:	learn: 2.7388337	total: 65ms	remaining: 1m 50s
1:	learn: 2.5308559	total: 123ms	remaining: 1m 44s
2:	learn: 2.3724458	total: 184ms	remaining: 1m 43s
3:	learn: 2.2442414	total: 252ms	remaining: 1m 46s
4:	learn: 2.1353905	total: 312ms	remaining: 1m 45s
5:	learn: 2.0414317	total: 374ms	remaining: 1m 45s
6:	learn: 1.9590249	total: 434ms	remaining: 1m 45s
7:	learn: 1.8856869	total: 493ms	remaining: 1m 44s
8:	learn: 1.8195862	total: 549ms	remaining: 1m 43s
9:	learn: 1.7592418	total: 602ms	remaining: 1m 41s
10:	learn: 1.7039015	total: 658ms	remaining: 1m 40s
11:	learn: 1.6533020	total: 719ms	remaining: 1m 41s
12:	learn: 1.6065661	total: 780ms	remaining: 1m 41s
13:	learn: 1.5629654	total: 835ms	remaining: 1m 40s
14:	learn: 1.5223289	total: 899ms	remaining: 1m 41s
15:	learn: 1.4843395	total: 971ms	remaining: 1m 42s
16:	learn: 1.4486344	total: 1.04s	remaining: 1m 42s
17:	learn: 1.4145901	total: 1.1s	remaining: 1m 42s
18:	learn: 1.3829351	total: 1.15s	remaining: 1m 42s
19:	learn: 1.3527791	tot

157:	learn: 0.5074407	total: 9.35s	remaining: 1m 31s
158:	learn: 0.5066229	total: 9.4s	remaining: 1m 31s
159:	learn: 0.5058778	total: 9.46s	remaining: 1m 31s
160:	learn: 0.5049955	total: 9.51s	remaining: 1m 30s
161:	learn: 0.5041314	total: 9.56s	remaining: 1m 30s
162:	learn: 0.5032886	total: 9.62s	remaining: 1m 30s
163:	learn: 0.5025945	total: 9.68s	remaining: 1m 30s
164:	learn: 0.5016370	total: 9.73s	remaining: 1m 30s
165:	learn: 0.5006945	total: 9.78s	remaining: 1m 30s
166:	learn: 0.4997743	total: 9.84s	remaining: 1m 30s
167:	learn: 0.4988640	total: 9.89s	remaining: 1m 30s
168:	learn: 0.4979464	total: 9.95s	remaining: 1m 30s
169:	learn: 0.4970817	total: 10s	remaining: 1m 30s
170:	learn: 0.4961189	total: 10.1s	remaining: 1m 29s
171:	learn: 0.4954583	total: 10.1s	remaining: 1m 29s
172:	learn: 0.4946112	total: 10.2s	remaining: 1m 29s
173:	learn: 0.4938150	total: 10.2s	remaining: 1m 29s
174:	learn: 0.4929270	total: 10.3s	remaining: 1m 29s
175:	learn: 0.4920476	total: 10.3s	remaining: 1m 

313:	learn: 0.4273371	total: 19.3s	remaining: 1m 25s
314:	learn: 0.4271136	total: 19.4s	remaining: 1m 25s
315:	learn: 0.4268857	total: 19.4s	remaining: 1m 25s
316:	learn: 0.4266015	total: 19.5s	remaining: 1m 25s
317:	learn: 0.4261231	total: 19.6s	remaining: 1m 24s
318:	learn: 0.4258076	total: 19.6s	remaining: 1m 24s
319:	learn: 0.4254418	total: 19.7s	remaining: 1m 24s
320:	learn: 0.4251345	total: 19.7s	remaining: 1m 24s
321:	learn: 0.4248679	total: 19.8s	remaining: 1m 24s
322:	learn: 0.4244008	total: 19.8s	remaining: 1m 24s
323:	learn: 0.4241027	total: 19.9s	remaining: 1m 24s
324:	learn: 0.4238641	total: 20s	remaining: 1m 24s
325:	learn: 0.4236649	total: 20s	remaining: 1m 24s
326:	learn: 0.4234966	total: 20.1s	remaining: 1m 24s
327:	learn: 0.4232373	total: 20.1s	remaining: 1m 24s
328:	learn: 0.4229951	total: 20.2s	remaining: 1m 24s
329:	learn: 0.4228171	total: 20.2s	remaining: 1m 24s
330:	learn: 0.4225702	total: 20.3s	remaining: 1m 23s
331:	learn: 0.4223092	total: 20.4s	remaining: 1m 2

469:	learn: 0.3910637	total: 28.4s	remaining: 1m 14s
470:	learn: 0.3909248	total: 28.5s	remaining: 1m 14s
471:	learn: 0.3907309	total: 28.5s	remaining: 1m 14s
472:	learn: 0.3905586	total: 28.6s	remaining: 1m 14s
473:	learn: 0.3902960	total: 28.6s	remaining: 1m 14s
474:	learn: 0.3901329	total: 28.7s	remaining: 1m 13s
475:	learn: 0.3899564	total: 28.7s	remaining: 1m 13s
476:	learn: 0.3898339	total: 28.8s	remaining: 1m 13s
477:	learn: 0.3896807	total: 28.9s	remaining: 1m 13s
478:	learn: 0.3895007	total: 28.9s	remaining: 1m 13s
479:	learn: 0.3893121	total: 29s	remaining: 1m 13s
480:	learn: 0.3891251	total: 29s	remaining: 1m 13s
481:	learn: 0.3889291	total: 29.1s	remaining: 1m 13s
482:	learn: 0.3887678	total: 29.2s	remaining: 1m 13s
483:	learn: 0.3885540	total: 29.2s	remaining: 1m 13s
484:	learn: 0.3883167	total: 29.3s	remaining: 1m 13s
485:	learn: 0.3881818	total: 29.3s	remaining: 1m 13s
486:	learn: 0.3880192	total: 29.4s	remaining: 1m 13s
487:	learn: 0.3879223	total: 29.4s	remaining: 1m 1

626:	learn: 0.3674554	total: 37.6s	remaining: 1m 4s
627:	learn: 0.3673205	total: 37.7s	remaining: 1m 4s
628:	learn: 0.3671985	total: 37.8s	remaining: 1m 4s
629:	learn: 0.3670655	total: 37.8s	remaining: 1m 4s
630:	learn: 0.3669810	total: 37.9s	remaining: 1m 4s
631:	learn: 0.3668860	total: 37.9s	remaining: 1m 4s
632:	learn: 0.3667655	total: 38s	remaining: 1m 4s
633:	learn: 0.3666039	total: 38s	remaining: 1m 3s
634:	learn: 0.3665442	total: 38.1s	remaining: 1m 3s
635:	learn: 0.3664265	total: 38.2s	remaining: 1m 3s
636:	learn: 0.3662680	total: 38.2s	remaining: 1m 3s
637:	learn: 0.3661094	total: 38.3s	remaining: 1m 3s
638:	learn: 0.3660116	total: 38.3s	remaining: 1m 3s
639:	learn: 0.3658544	total: 38.4s	remaining: 1m 3s
640:	learn: 0.3657323	total: 38.5s	remaining: 1m 3s
641:	learn: 0.3655466	total: 38.5s	remaining: 1m 3s
642:	learn: 0.3654819	total: 38.6s	remaining: 1m 3s
643:	learn: 0.3653260	total: 38.6s	remaining: 1m 3s
644:	learn: 0.3651638	total: 38.7s	remaining: 1m 3s
645:	learn: 0.36

...

945:	learn: 0.3411195	total: 57.9s	remaining: 46.1s
946:	learn: 0.3410106	total: 57.9s	remaining: 46.1s
947:	learn: 0.3409748	total: 58s	remaining: 46s
948:	learn: 0.3409101	total: 58s	remaining: 45.9s
949:	learn: 0.3407929	total: 58.1s	remaining: 45.9s
950:	learn: 0.3407242	total: 58.2s	remaining: 45.8s
951:	learn: 0.3406769	total: 58.2s	remaining: 45.7s
952:	learn: 0.3406229	total: 58.3s	remaining: 45.7s
953:	learn: 0.3405518	total: 58.3s	remaining: 45.6s
954:	learn: 0.3405028	total: 58.4s	remaining: 45.5s
955:	learn: 0.3404671	total: 58.4s	remaining: 45.5s
956:	learn: 0.3404157	total: 58.5s	remaining: 45.4s
957:	learn: 0.3403304	total: 58.5s	remaining: 45.3s
958:	learn: 0.3402874	total: 58.6s	remaining: 45.3s
959:	learn: 0.3401702	total: 58.7s	remaining: 45.2s
960:	learn: 0.3400941	total: 58.7s	remaining: 45.2s
961:	learn: 0.3400697	total: 58.8s	remaining: 45.1s
962:	learn: 0.3399999	total: 58.9s	remaining: 45s
963:	learn: 0.3399588	total: 58.9s	remaining: 45s
964:	learn: 0.3399044	

1103:	learn: 0.3314605	total: 1m 6s	remaining: 35.8s
1104:	learn: 0.3314215	total: 1m 6s	remaining: 35.8s
1105:	learn: 0.3313774	total: 1m 6s	remaining: 35.7s
1106:	learn: 0.3313345	total: 1m 6s	remaining: 35.6s
1107:	learn: 0.3313172	total: 1m 6s	remaining: 35.6s
1108:	learn: 0.3312718	total: 1m 6s	remaining: 35.5s
1109:	learn: 0.3312333	total: 1m 6s	remaining: 35.4s
1110:	learn: 0.3311446	total: 1m 6s	remaining: 35.4s
1111:	learn: 0.3310997	total: 1m 6s	remaining: 35.3s
1112:	learn: 0.3310932	total: 1m 6s	remaining: 35.2s
1113:	learn: 0.3310464	total: 1m 6s	remaining: 35.2s
1114:	learn: 0.3309955	total: 1m 6s	remaining: 35.1s
1115:	learn: 0.3309241	total: 1m 6s	remaining: 35s
1116:	learn: 0.3309037	total: 1m 6s	remaining: 35s
1117:	learn: 0.3308424	total: 1m 7s	remaining: 34.9s
1118:	learn: 0.3307492	total: 1m 7s	remaining: 34.8s
1119:	learn: 0.3306417	total: 1m 7s	remaining: 34.8s
1120:	learn: 0.3305526	total: 1m 7s	remaining: 34.7s
1121:	learn: 0.3305226	total: 1m 7s	remaining: 34.

1257:	learn: 0.3231736	total: 1m 14s	remaining: 26.2s
1258:	learn: 0.3231414	total: 1m 14s	remaining: 26.1s
1259:	learn: 0.3231031	total: 1m 14s	remaining: 26s
1260:	learn: 0.3230316	total: 1m 14s	remaining: 26s
1261:	learn: 0.3229855	total: 1m 14s	remaining: 25.9s
1262:	learn: 0.3229454	total: 1m 14s	remaining: 25.9s
1263:	learn: 0.3228709	total: 1m 14s	remaining: 25.8s
1264:	learn: 0.3228412	total: 1m 14s	remaining: 25.7s
1265:	learn: 0.3227678	total: 1m 14s	remaining: 25.7s
1266:	learn: 0.3227326	total: 1m 14s	remaining: 25.6s
1267:	learn: 0.3227087	total: 1m 14s	remaining: 25.6s
1268:	learn: 0.3226640	total: 1m 15s	remaining: 25.5s
1269:	learn: 0.3226229	total: 1m 15s	remaining: 25.4s
1270:	learn: 0.3225769	total: 1m 15s	remaining: 25.4s
1271:	learn: 0.3225018	total: 1m 15s	remaining: 25.3s
1272:	learn: 0.3224686	total: 1m 15s	remaining: 25.3s
1273:	learn: 0.3224226	total: 1m 15s	remaining: 25.2s
1274:	learn: 0.3223558	total: 1m 15s	remaining: 25.1s
1275:	learn: 0.3223200	total: 1m

1410:	learn: 0.3161010	total: 1m 22s	remaining: 16.9s
1411:	learn: 0.3160553	total: 1m 22s	remaining: 16.8s
1412:	learn: 0.3160302	total: 1m 22s	remaining: 16.7s
1413:	learn: 0.3159992	total: 1m 22s	remaining: 16.7s
1414:	learn: 0.3159802	total: 1m 22s	remaining: 16.6s
1415:	learn: 0.3159225	total: 1m 22s	remaining: 16.6s
1416:	learn: 0.3158716	total: 1m 22s	remaining: 16.5s
1417:	learn: 0.3158430	total: 1m 22s	remaining: 16.4s
1418:	learn: 0.3158172	total: 1m 22s	remaining: 16.4s
1419:	learn: 0.3157605	total: 1m 22s	remaining: 16.3s
1420:	learn: 0.3156951	total: 1m 22s	remaining: 16.3s
1421:	learn: 0.3156810	total: 1m 22s	remaining: 16.2s
1422:	learn: 0.3156356	total: 1m 22s	remaining: 16.1s
1423:	learn: 0.3155639	total: 1m 22s	remaining: 16.1s
1424:	learn: 0.3155122	total: 1m 22s	remaining: 16s
1425:	learn: 0.3154581	total: 1m 23s	remaining: 16s
1426:	learn: 0.3154191	total: 1m 23s	remaining: 15.9s
1427:	learn: 0.3153456	total: 1m 23s	remaining: 15.8s
1428:	learn: 0.3152972	total: 1m

1563:	learn: 0.3098908	total: 1m 30s	remaining: 7.87s
1564:	learn: 0.3098432	total: 1m 30s	remaining: 7.82s
1565:	learn: 0.3098018	total: 1m 30s	remaining: 7.76s
1566:	learn: 0.3097956	total: 1m 30s	remaining: 7.7s
1567:	learn: 0.3097174	total: 1m 30s	remaining: 7.64s
1568:	learn: 0.3096664	total: 1m 31s	remaining: 7.61s
1569:	learn: 0.3096172	total: 1m 31s	remaining: 7.55s
1570:	learn: 0.3095873	total: 1m 31s	remaining: 7.49s
1571:	learn: 0.3095616	total: 1m 31s	remaining: 7.43s
1572:	learn: 0.3095207	total: 1m 31s	remaining: 7.38s
1573:	learn: 0.3094865	total: 1m 31s	remaining: 7.32s
1574:	learn: 0.3094566	total: 1m 31s	remaining: 7.26s
1575:	learn: 0.3094267	total: 1m 31s	remaining: 7.2s
1576:	learn: 0.3094161	total: 1m 31s	remaining: 7.14s
1577:	learn: 0.3093701	total: 1m 31s	remaining: 7.08s
1578:	learn: 0.3093266	total: 1m 31s	remaining: 7.03s
1579:	learn: 0.3092849	total: 1m 31s	remaining: 6.97s
1580:	learn: 0.3092711	total: 1m 31s	remaining: 6.91s
1581:	learn: 0.3092482	total: 

In [242]:
calib_mc = mli.SplineCalib(random_state=42) #SplineCalibratedClassifierCV
calib_mc.fit(cv_train_preds, y, verbose=True)

Calibrating Class 0
Calibrating Class 1
Calibrating Class 2
Calibrating Class 3
Calibrating Class 4
Calibrating Class 5
Calibrating Class 6
Calibrating Class 7
Calibrating Class 8
Calibrating Class 9
Calibrating Class 10
Calibrating Class 11
Calibrating Class 12
Calibrating Class 13
Calibrating Class 14
Calibrating Class 15
Calibrating Class 16
Calibrating Class 17
Calibrating Class 18
Calibrating Class 19
Calibrating Class 20


In [243]:
test_probs_calibrated = calib_mc.calibrate(proba)
y_test = pd.DataFrame(test_probs_calibrated)
y_test.columns = le.inverse_transform(y_test.columns)

### Submit

In [244]:
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + c, y_test[c].iloc[i]])

df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0

In [245]:
mask = df_answer['ID X PCODE'].str.contains('GYSR')
column_name = 'Label'
df_answer.loc[mask, column_name] = 0

In [246]:
df_answer.reset_index(drop=True, inplace=True)
df_answer.to_csv('D:\Python\Zindi\Insuranse\submissioninsur83lensemblevotesplinerandstate42.csv', index=False)